In [1]:
import json
import random

In [2]:
CNC_METADATA_PATH = "../cnc-images/metadata/dungeons.json"
GEO_LAND_PATH = "../datasources_geojson/crypts_land_order.geojson"
GEO_SEA_PATH = "../datasources_geojson/crypts_sea.geojson"

In [3]:
def load_json(path):
    with open(path, "r") as file:
        data = json.load(file)
    return data

cnc_data = load_json(CNC_METADATA_PATH)
land_data = load_json(GEO_LAND_PATH)
sea_data = load_json(GEO_SEA_PATH)

random.seed(42)
random.shuffle(land_data["features"])

In [4]:
nones = []
environments = []
affinities = []

for cidx, crypt in enumerate(cnc_data["dungeons"]):
    if crypt is None:
        nones.append(cidx)
        continue
    env, aff = crypt["environment"], crypt["affinity"]
    if env not in environments:
        environments.append(env)
    if aff not in affinities:
        affinities.append(aff)

In [5]:
len(nones)

227

In [6]:
environments

['Stone Temple',
 'Mountain Deep',
 'Desert Oasis',
 'Underwater Keep',
 'Forest Ruins',
 "Ember's Glow"]

In [7]:
affinities

['Sorrow',
 'Needles',
 'the Storm',
 None,
 'Agony',
 'Mire',
 'Honour',
 'the Plague',
 'Despair',
 'Blight',
 'Havoc',
 'the Titans',
 'the Stars',
 'Fire',
 'Pine',
 'Horror',
 'Pandemonium',
 'Dusk',
 'the Apocalypse',
 'the Song',
 'Woe',
 'Corruption',
 'Bone',
 'the Willows',
 'the Viper',
 'the Golem',
 'the Fox',
 'the Twins',
 'Torment',
 'the Beast',
 'the Kraken',
 'the Tear',
 'the Vortex',
 'Wisdom',
 'Enlightenment',
 'Mist',
 'Victory',
 'the Brood',
 'Loathing',
 'the Behemoth',
 'Hate',
 'Giants',
 'Rage',
 'Pain',
 'Vengeance',
 'Fury',
 'Brimstone',
 'the Gale',
 'Rapture',
 'Brilliance',
 'Sand',
 'Dread',
 'the Leech',
 'the Moon',
 'the Wisp',
 'the Sun',
 'Gloom',
 'Anger',
 'the Phoenix',
 'the Root']

In [8]:
orders = [
    "Rage",
    "Detection",
    "Anger",
    "Giants",
    "Skill",
    "Enlightenment",
    "Power",
    "Brilliance",
    "Perfection",
    "Titans",
    "Fury",
    "Reflection",
    "the Twins",
    "Vitriol",
    "the Fox",
    "Protection"
]

## sea

In [9]:
sea_crypts = []

for idx, crypt in enumerate(cnc_data["dungeons"]):
    if crypt is None:
        continue
    if crypt["environment"] == "Underwater Keep":
        sea_crypts.append(crypt)
        cnc_data["dungeons"][idx] = None
        
for idx, co in enumerate(sea_data["features"]):
    co["properties"].update(sea_crypts[idx])

## land

In [10]:
ordered_crypts = {}
other_crypts = []
for idx, crypt in enumerate(cnc_data["dungeons"]):
    if crypt is None:
        continue
    order = crypt["affinity"]
    if order in orders and order != "Underwater Keep":
        if order not in ordered_crypts:
            ordered_crypts[order] = [crypt]
        else:
            ordered_crypts[order].append(crypt)
    else:
        other_crypts.append(crypt)
            
ordered_crypts

{'the Fox': [{'tokenId': 82,
   'size': 24,
   'layout': '0x0000003be0003bffe03f00203800203800207800207f60207fe0207f00207f60207fe0207f7e207f7e207ffe207ffe207ffe201feff01ffff07ffff07ffff0007070000000000000000000000000000000000000000000000000000000000000',
   'environment': 'Desert Oasis',
   'name': 'Whispering Crypt of the Fox',
   'numDoors': 1,
   'numPoints': 3,
   'affinity': 'the Fox',
   'legendary': 0,
   'structure': 0},
  {'tokenId': 141,
   'size': 16,
   'layout': '0x000000001c001ff8087808003c003c003fe03fe07fe07f201cf01ff0000000000000000000000000000000000000000000000000000000000000000000000000',
   'environment': 'Stone Temple',
   'name': 'Hidden Cloister of the Fox',
   'numDoors': 1,
   'numPoints': 2,
   'affinity': 'the Fox',
   'legendary': 0,
   'structure': 0},
  {'tokenId': 267,
   'size': 20,
   'layout': '0x00000000000000000000000003e0003e0003ffc03e3f0083d00bc100bf107bf107ff907fffc7f07c7f07c1807c00000000000000000000000000000000000000',
   'environment': 'Forest Ru

In [11]:
for order, prio_crypts in ordered_crypts.items():
    for prio_crypt in prio_crypts:
        for cidx, feature in enumerate(land_data["features"]):
            if feature["properties"]["order_name"] == order and len(feature["properties"]) == 2:
                land_data["features"][cidx]["properties"].update(prio_crypt)
                break

In [12]:
for sec_crypt in other_crypts:
    for cidx, feature in enumerate(land_data["features"]):
        if len(feature["properties"]) == 2:
            land_data["features"][cidx]["properties"].update(sec_crypt)
            break

In [13]:
with open("../datasources_json/crypts_sea.json", "w") as file:
    json.dump(sea_data, file)
with open("../datasources_json/crypts_land.json", "w") as file:
    json.dump(land_data, file)

## combined

In [14]:
all_features = sea_data["features"] + land_data["features"]
print(len(all_features))
all_features = [x for x in all_features if len(x["properties"]) > 2]
print(len(all_features))

combined = {'type': 'FeatureCollection',
    'name': 'crypts_sea',
    'crs': {
        'type': 'name',
        'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}
    },
    'features': all_features
}

8800
8772


In [15]:
with open("../datasources_json/crypts_all.json", "w") as file:
    json.dump(combined, file)

In [18]:
len(combined["features"])

8772

In [19]:
idx = [x["properties"]["tokenId"] for x in combined["features"]]

In [21]:
x = [x for x in range(9000) if x not in idx]

In [22]:
x

[0,
 270,
 685,
 1135,
 1807,
 3032,
 4706,
 5947,
 6421,
 7162,
 7730,
 7785,
 7786,
 7787,
 7788,
 7789,
 7790,
 7791,
 7792,
 7793,
 7794,
 7795,
 7796,
 7797,
 7798,
 7799,
 7800,
 7801,
 7802,
 7803,
 7804,
 7805,
 7806,
 7807,
 7808,
 7809,
 7810,
 7811,
 7812,
 7813,
 7814,
 7815,
 7816,
 7817,
 7818,
 7819,
 7820,
 7821,
 7822,
 7823,
 7824,
 7825,
 7826,
 7827,
 7828,
 7829,
 7830,
 7831,
 7832,
 7833,
 7834,
 7835,
 7836,
 7837,
 7838,
 7839,
 7840,
 7841,
 7842,
 7843,
 7844,
 7845,
 7846,
 7847,
 7848,
 7849,
 7850,
 7851,
 7852,
 7853,
 7854,
 7855,
 7856,
 7857,
 7858,
 7859,
 7860,
 7861,
 7862,
 7863,
 7864,
 7865,
 7866,
 7867,
 7868,
 7869,
 7870,
 7871,
 7872,
 7873,
 7874,
 7875,
 7876,
 7877,
 7878,
 7879,
 7880,
 7881,
 7882,
 7883,
 7884,
 7885,
 7886,
 7887,
 7888,
 7889,
 7890,
 7891,
 7892,
 7893,
 7894,
 7895,
 7896,
 7897,
 7898,
 7899,
 7900,
 7901,
 7902,
 7903,
 7904,
 7905,
 7906,
 7907,
 7908,
 7909,
 7910,
 7911,
 7912,
 7913,
 7914,
 7915,
 7916,
 791